# Preparación de datasets

In [1]:
# importacion general de librerias
import pandas as pd

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
#https://drive.google.com/file/d/1y4_n4twjE4VSLSC-V7QsdPJ5JR9fiVgG/view?usp=sharing GooglePlayStore.csv
#https://drive.google.com/file/d/1HBgBzv4HU1wQKKblj8p--9lJEMSGYczg/view?usp=sharing GooglePlayStore_User_Reviews.csv

id1='1y4_n4twjE4VSLSC-V7QsdPJ5JR9fiVgG'
id2='1HBgBzv4HU1wQKKblj8p--9lJEMSGYczg'
downloaded1 = drive.CreateFile({'id': id1})
downloaded1.GetContentFile('GooglePlayStore.csv')
downloaded2 = drive.CreateFile({'id': id2})
downloaded2.GetContentFile('GooglePlayStore_User_Reviews.csv')

Se obtuvieron únicamente las columnas necesarias para la resolución de los problemas en cuestión.

In [5]:
appDetails=pd.read_csv('GooglePlayStore.csv', usecols=['App','Category'])
appReviews=pd.read_csv('GooglePlayStore_User_Reviews.csv', usecols=['App', 'Sentiment'])

In [6]:
appReviews

,App,Sentiment
0,10 Best Foods for You,Positive
1,10 Best Foods for You,Positive
2,10 Best Foods for You,NaN
3,10 Best Foods for You,Positive
4,10 Best Foods for You,Positive
...,...,...
64290,Houzz Interior Design Ideas,NaN
64291,Houzz Interior Design Ideas,NaN
64292,Houzz Interior Design Ideas,NaN
64293,Houzz Interior Design Ideas,NaN


In [7]:
appReviews['App'].value_counts(dropna=False)

Angry Birds Classic                                  320
CBS Sports App - Scores, News, Stats & Watch Live    320
Bowmasters                                           320
8 Ball Pool                                          300
Helix Jump                                           300
                                                    ... 
Easy Healthy Recipes                                  31
Dresses Ideas & Fashions +3000                        31
Detector de Radares Gratis                            31
Easy Hair Style Design                                30
Drawing Clothes Fashion Ideas                         30
Name: App, Length: 1074, dtype: int64

In [8]:
appReviews['App'].value_counts(dropna=True)

Angry Birds Classic                                  320
CBS Sports App - Scores, News, Stats & Watch Live    320
Bowmasters                                           320
8 Ball Pool                                          300
Helix Jump                                           300
                                                    ... 
Easy Healthy Recipes                                  31
Dresses Ideas & Fashions +3000                        31
Detector de Radares Gratis                            31
Easy Hair Style Design                                30
Drawing Clothes Fashion Ideas                         30
Name: App, Length: 1074, dtype: int64

Es posible observar que la columna App no tiene valores null.

In [9]:
appReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64295 entries, 0 to 64294
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   App        64295 non-null  object
 1   Sentiment  37432 non-null  object
dtypes: object(2)
memory usage: 1004.7+ KB


Considero que los reviews sin sentimiento no cobran sentido:

In [10]:
validAppReviews = appReviews[appReviews['Sentiment'].notna()]
validAppReviews

,App,Sentiment
0,10 Best Foods for You,Positive
1,10 Best Foods for You,Positive
3,10 Best Foods for You,Positive
4,10 Best Foods for You,Positive
5,10 Best Foods for You,Positive
...,...,...
64222,Housing-Real Estate & Property,Positive
64223,Housing-Real Estate & Property,Positive
64226,Housing-Real Estate & Property,Negative
64227,Housing-Real Estate & Property,Positive


In [11]:
validAppReviews['App']=validAppReviews['App'].astype('string')
validAppReviews['Sentiment']=validAppReviews['Sentiment'].astype('category')
validAppReviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37432 entries, 0 to 64230
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   App        37432 non-null  string  
 1   Sentiment  37432 non-null  category
dtypes: category(1), string(1)
memory usage: 621.6 KB


In [12]:
appDetails

,App,Category
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN
1,Coloring book moana,ART_AND_DESIGN
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN
3,Sketch - Draw & Paint,ART_AND_DESIGN
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN
...,...,...
10836,Sya9a Maroc - FR,FAMILY
10837,Fr. Mike Schmitz Audio Teachings,FAMILY
10838,Parkinson Exercices FR,MEDICAL
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE


In [13]:
appDetails.duplicated().value_counts()

False    9745
True     1096
dtype: int64

Se droppearon las aplicaciones con el mismo valor en todas las columnas, dado que podrían existir aplicaciones con el mismo nombre.

In [14]:
validAppDetails = appDetails.drop_duplicates()
validAppDetails

,App,Category
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN
1,Coloring book moana,ART_AND_DESIGN
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN
3,Sketch - Draw & Paint,ART_AND_DESIGN
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN
...,...,...
10836,Sya9a Maroc - FR,FAMILY
10837,Fr. Mike Schmitz Audio Teachings,FAMILY
10838,Parkinson Exercices FR,MEDICAL
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE


Las aplicaciones sin nombre no son válidas:

In [15]:
validAppDetails = validAppDetails[appDetails['App'].notna()]
validAppDetails

,App,Category
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN
1,Coloring book moana,ART_AND_DESIGN
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN
3,Sketch - Draw & Paint,ART_AND_DESIGN
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN
...,...,...
10836,Sya9a Maroc - FR,FAMILY
10837,Fr. Mike Schmitz Audio Teachings,FAMILY
10838,Parkinson Exercices FR,MEDICAL
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE


In [16]:
validAppDetails.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9745 entries, 0 to 10840
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   App       9745 non-null   object
 1   Category  9745 non-null   object
dtypes: object(2)
memory usage: 228.4+ KB


In [17]:
validAppDetails['Category'] = validAppDetails['Category'].str.replace('_',' ')
validAppDetails['Category'] = validAppDetails['Category'].str.lower()
validAppDetails

,App,Category
0,Photo Editor & Candy Camera & Grid & ScrapBook,art and design
1,Coloring book moana,art and design
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",art and design
3,Sketch - Draw & Paint,art and design
4,Pixel Draw - Number Art Coloring Book,art and design
...,...,...
10836,Sya9a Maroc - FR,family
10837,Fr. Mike Schmitz Audio Teachings,family
10838,Parkinson Exercices FR,medical
10839,The SCP Foundation DB fr nn5n,books and reference


In [18]:
validAppDetails['Category'].unique()

array(['art and design', 'auto and vehicles', 'beauty',
       'books and reference', 'business', 'comics', 'communication',
       'dating', 'education', 'entertainment', 'events', 'finance',
       'food and drink', 'health and fitness', 'house and home',
       'libraries and demo', 'lifestyle', 'game', 'family', 'medical',
       'social', 'shopping', 'photography', 'sports', 'travel and local',
       'tools', 'personalization', 'productivity', 'parenting', 'weather',
       'video players', 'news and magazines', 'maps and navigation',
       '1.9'], dtype=object)

A la hora de formatear y revisar las categorías se detectó una anomalía: categoría "1.9". Fue posible observar que la aplicación en cuestión contenía datos inválidos en todas las columnas, por lo que fue droppeada.

In [19]:
validAppDetails[validAppDetails['Category']=='1.9']

,App,Category
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9


In [20]:
validAppDetails = validAppDetails.drop(10472)
validAppDetails

,App,Category
0,Photo Editor & Candy Camera & Grid & ScrapBook,art and design
1,Coloring book moana,art and design
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",art and design
3,Sketch - Draw & Paint,art and design
4,Pixel Draw - Number Art Coloring Book,art and design
...,...,...
10836,Sya9a Maroc - FR,family
10837,Fr. Mike Schmitz Audio Teachings,family
10838,Parkinson Exercices FR,medical
10839,The SCP Foundation DB fr nn5n,books and reference


Cambiamos el datatype de algunas columnas para mejorar el rendimiento.

In [21]:
validAppDetails['App']=validAppDetails['App'].astype('string')
validAppDetails['Category']=validAppDetails['Category'].astype('category')
validAppDetails.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9744 entries, 0 to 10840
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   App       9744 non-null   string  
 1   Category  9744 non-null   category
dtypes: category(1), string(1)
memory usage: 163.1 KB


# Ejercicio 21
Para cada categoría, indicar cuál es la aplicación que tiene mayor cantidad de reviews con sentimiento negativo (⭐⭐)

In [22]:
negativeAppReviews = validAppReviews.loc[:,['App','Sentiment']]
negativeAppReviews = negativeAppReviews[negativeAppReviews['Sentiment']=='Negative']
negativeAppReviews

,App,Sentiment
32,10 Best Foods for You,Negative
43,10 Best Foods for You,Negative
68,10 Best Foods for You,Negative
85,10 Best Foods for You,Negative
95,10 Best Foods for You,Negative
...,...,...
64215,Housing-Real Estate & Property,Negative
64216,Housing-Real Estate & Property,Negative
64220,Housing-Real Estate & Property,Negative
64226,Housing-Real Estate & Property,Negative


In [23]:
negativeReviewsPerApp = negativeAppReviews.groupby(['App']).size().reset_index(name='Negative Reviews')
negativeReviewsPerApp

,App,Negative Reviews
0,10 Best Foods for You,10
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,1
2,11st,7
3,1800 Contacts - Lens Store,6
4,1LINE – One Line with One Touch,8
...,...,...
777,Hotels.com: Book Hotel Rooms & Find Vacation D...,21
778,Hotspot Shield Free VPN Proxy & Wi-Fi Security,3
779,Hotstar,14
780,Hotwire Hotel & Car Rental App,6


In [24]:
appCategory = validAppDetails.loc[:, ['App','Category']]
appCategory

,App,Category
0,Photo Editor & Candy Camera & Grid & ScrapBook,art and design
1,Coloring book moana,art and design
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",art and design
3,Sketch - Draw & Paint,art and design
4,Pixel Draw - Number Art Coloring Book,art and design
...,...,...
10836,Sya9a Maroc - FR,family
10837,Fr. Mike Schmitz Audio Teachings,family
10838,Parkinson Exercices FR,medical
10839,The SCP Foundation DB fr nn5n,books and reference


Pueden existir categorías donde la mayor cantidad de reviews negativos es 0, por lo que no se debe utilizar how='inner'(el default), sino tomar de referencia el df con todas las apps y rellenar NaNs con 0. Otra forma es utilizar how='inner' y luego llenar a mano las categorías que faltan(si es que las hay) con cualquier app que tendrá 0 reviews negativos.

In [28]:
negativeReviewsPerAppWithCategory = pd.merge(appCategory, negativeReviewsPerApp, how='left')
negativeReviewsPerAppWithCategory['Negative Reviews'] = negativeReviewsPerAppWithCategory['Negative Reviews'].fillna(0).astype('int')
negativeReviewsPerAppWithCategory

,App,Category,Negative Reviews
0,Photo Editor & Candy Camera & Grid & ScrapBook,art and design,0
1,Coloring book moana,art and design,14
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",art and design,0
3,Sketch - Draw & Paint,art and design,0
4,Pixel Draw - Number Art Coloring Book,art and design,0
...,...,...,...
9739,Sya9a Maroc - FR,family,0
9740,Fr. Mike Schmitz Audio Teachings,family,0
9741,Parkinson Exercices FR,medical,0
9742,The SCP Foundation DB fr nn5n,books and reference,0


Para cada categoría, la aplicación que tiene mayor cantidad de reviews con sentimiento negativo:

In [29]:
indexMostNegativeReviewsPerCategory = negativeReviewsPerAppWithCategory.groupby(['Category'])['Negative Reviews'].idxmax()
AppsWithMostNegativeReviewsPerCategory = negativeReviewsPerAppWithCategory.loc[indexMostNegativeReviewsPerCategory]
AppsWithMostNegativeReviewsPerCategory.rename(columns = {'App':'App with Most Negative Reviews'}, inplace = True)
AppsWithMostNegativeReviewsPerCategory.set_index('Category')

,App with Most Negative Reviews,Negative Reviews
Category,,
art and design,Coloring book moana,14
auto and vehicles,Free VIN Report for Used Cars,3
beauty,"BestCam Selfie-selfie, beauty camera, photo ed...",32
books and reference,Amazon Kindle,33
business,HipChat - Chat Built for Teams,26
comics,DC Comics,1
communication,Azar,25
dating,Free Dating App - Meet Local Singles - Flirt Chat,31
education,Duolingo: Learn Languages Free,34
